# AIM : TO FIND THE BEST NEIGHBORHOODS WHERE A BUSINESS OWNER CAN OPEN AN INDIAN RESTAURANT

In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import json
import matplotlib.cm as cm
import matplotlib.colors as colors

## Dataset for the neighborhoods of Delhi obtained from Kaggle.com
## https://www.kaggle.com/shaswatd673/delhi-neighborhood-data/data?select=delhi_dataSet.csv

## CLEANING THE DATASET

In [2]:
df_delhi = pd.read_csv("Delhi_dataset.csv")
df_delhi

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
4,4,North West Delhi,Begum Pur,NaN,NaN
...,...,...,...,...,...
180,180,West Delhi,Tihar Village,NaN,NaN
181,181,West Delhi,Tilak Nagar,28.639650,77.094039
182,182,West Delhi,Uttam Nagar,NaN,NaN
183,183,West Delhi,Vikas Nagar,28.644009,77.054470


In [3]:
df_delhi.isnull().sum()

Unnamed: 0       0
Borough          0
Neighborhood     0
latitude        22
longitude       22
dtype: int64

In [4]:
df_delhi.shape

(185, 5)

In [5]:
df1 = df_delhi.dropna(axis=0)
df1.head()

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
5,5,North West Delhi,Dhaka,39.031714,-90.261223


In [6]:
df1.shape

(163, 5)

In [7]:
df1.drop(df1.columns[df1.columns.str.contains('unnamed',case=False)],axis=1,inplace=True)
df1.head()

C:\Users\Vaishali\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
5,North West Delhi,Dhaka,39.031714,-90.261223


## USING GEOLOCATOR TO FIND COORDINATES OF DELHI AND MAPPING IT USING FOLIUM

In [8]:
address = 'Delhi , IN'
geolocator = Nominatim(user_agent="pulkit")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print(lat,lon)

28.6517178 77.2219388


In [9]:
map_del = folium.Map(location=[lat, lon], zoom_start=10)

for lat, lng, borough, Neighborhood in zip(df1['latitude'], df1['longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_del)
map_del

## USING THE FOURSQUARE API TO GET VENUES IN A RADIUS OF 1000m OF THE NEIGHBORHOODS

In [10]:
CLIENT_ID = 'LQTRG5MA4F4VIYSEM4XXWH531301VZJCO3XBJVBMIGQXBFJ0' # your Foursquare ID
CLIENT_SECRET = 'GRHG113PBGAJQPW5E4Q4JYWYVRNBFAJQMY135DRQOE01WD1R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LQTRG5MA4F4VIYSEM4XXWH531301VZJCO3XBJVBMIGQXBFJ0
CLIENT_SECRET:GRHG113PBGAJQPW5E4Q4JYWYVRNBFAJQMY135DRQOE01WD1R


In [11]:
radius=1000
LIMIT=100

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return nearby_venues

In [13]:
import requests
del_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['latitude'],
                                   longitudes=df1['longitude']
                                  )

Adarsh Nagar
Ashok Vihar
Azadpur
Bawana
Dhaka
Jahangirpuri
Karala
Keshav Puram
Kingsway Camp
Kohat Enclave
Model Town
Narela
Pitam Pura
Rani Bagh
Rithala
Shalimar Bagh
Shakti Nagar
Bara Hindu Rao
Chandni Chowk
Chawri Bazaar
Civil Lines
Dariba Kalan
Kamla Nagar
Kashmiri Gate
Kotwali
Lahori Gate
Nehru Vihar
Paharganj
Pratap Nagar
Pul Bangash
Roshanara Bagh
Sangam Vihar
Sant Nagar
Sarai Kale Khan
Sarai Rohilla
Shakti Nagar
Shastri Nagar
Timarpur
Wazirabad
Babarpur
Dilshad Garden
Kabir Nagar
Mandoli
Maujpur
Nand Nagri
Naveen Shahdara
Shahdara
Sonia Vihar
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Bara Hindu Rao
Paharganj
Barakhamba Road
Chanakyapuri
Connaught Place
Gautampuri
Gole Market
Golf Links
INA Colony
Khan Market
Pragati Maidan
Brij Puri
East Vinod Nagar
Gandhi Nagar
Geeta Colony
Jhilmil Colony
Krishna Nagar
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Patparganj
Preet Vihar
Anand Vihar
Shakarpur
Shastri Nagar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Shahd

In [14]:
del_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614192,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Adarsh Nagar,28.614192,77.071541,Uttam nagar,28.620201,77.068709,Metro Station
2,Adarsh Nagar,28.614192,77.071541,Gold's Gym A Block Janakpuri,28.622439,77.069348,Gym
3,Adarsh Nagar,28.614192,77.071541,Potholes at Dabri,28.605309,77.072504,Pool
4,Ashok Vihar,28.699453,77.184826,Bellagio,28.696361,77.180021,Asian Restaurant


In [15]:
pd.set_option('display.max_rows', 1000)
del_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,45,45,45,45,45,45
Accessories Store,3,3,3,3,3,3
Airport Terminal,2,2,2,2,2,2
American Restaurant,14,14,14,14,14,14
Antique Shop,2,2,2,2,2,2
Arcade,18,18,18,18,18,18
Art Gallery,6,6,6,6,6,6
Art Museum,1,1,1,1,1,1
Arts & Crafts Store,6,6,6,6,6,6


In [16]:
#here we see that there are 316 indian restaurants. 

In [17]:
del_onehot = pd.get_dummies(del_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
del_onehot['Neighborhood'] = del_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [del_onehot.columns[-1]] + list(del_onehot.columns[:-1])
del_onehot.head()

,ATM,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
del_grouped = del_onehot.groupby('Neighborhood').mean().reset_index()
del_grouped

,Neighborhood,ATM,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,Alaknanda,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Anand Vihar,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ashok Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,Ashok Vihar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,Azadpur,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,Babarpur,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,Badarpur,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,Bali Nagar,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bara Hindu Rao,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
del_grouped.columns

Index(['Neighborhood', 'ATM', 'Accessories Store', 'Airport Terminal',
       'American Restaurant', 'Antique Shop', 'Arcade', 'Art Gallery',
       'Art Museum', 'Arts & Crafts Store',
       ...
       'University', 'Vegetarian / Vegan Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Warehouse Store', 'Whisky Bar', 'Wine Bar',
       'Wings Joint', 'Women's Store', 'Yoga Studio'],
      dtype='object', length=206)

## SEGREGATING THE DATASET WHICH ONLY INCLUDES THE DATA FOR INDIAN RESTAURANTS.

In [20]:
del_ir = del_grouped[["Neighborhood","Indian Restaurant"]]
del_ir.head()

,Neighborhood,Indian Restaurant
0,Adarsh Nagar,0.250000
1,Alaknanda,0.076923
2,Anand Vihar,0.153846
3,Ashok Nagar,0.000000
4,Ashok Vihar,0.111111


## APPLYING KMEANS CLUSTERING

In [21]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

del_clustering = del_ir.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(del_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 3, 1, 2, 3, 0, 3, 2])

In [22]:
del_merged = del_ir.copy()
del_merged["Cluster Labels"] = kmeans.labels_
del_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels
0,Adarsh Nagar,0.250000,2
1,Alaknanda,0.076923,1
2,Anand Vihar,0.153846,2
3,Ashok Nagar,0.000000,3
4,Ashok Vihar,0.111111,1


In [23]:
del_merged = del_merged.join(df1.set_index("Neighborhood"), on="Neighborhood")

print(del_merged.shape)
del_merged.head()

(162, 6)


,Neighborhood,Indian Restaurant,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,0.250000,2,North West Delhi,28.614192,77.071541
1,Alaknanda,0.076923,1,South Delhi,28.529336,77.251632
2,Anand Vihar,0.153846,2,East Delhi,28.625691,77.101941
3,Ashok Nagar,0.000000,3,West Delhi,39.031714,-90.261223
4,Ashok Vihar,0.111111,1,North West Delhi,28.699453,77.184826


In [24]:
map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(del_merged['latitude'], del_merged['longitude'], del_merged['Neighborhood'], del_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## EXAMINING THE CLUSTERS

In [25]:
dx0 =del_merged.loc[del_merged['Cluster Labels'] == 0]
dx0.head(10)

,Neighborhood,Indian Restaurant,Cluster Labels,Borough,latitude,longitude
7,Badarpur,0.400000,0,South Delhi,28.503638,77.301976
15,Chawri Bazaar,0.318182,0,North Delhi,28.649927,77.229788
21,Daryaganj,0.312500,0,Central Delhi,28.646090,77.243048
33,Gautampuri,0.400000,0,New Delhi,28.511570,77.302623
99,Okhla,0.428571,0,South Delhi,28.563662,77.289055
106,Pitam Pura,0.400000,0,North West Delhi,28.703268,77.132250


In [26]:
dx0.mean()

Indian Restaurant     0.376542
Cluster Labels        0.000000
latitude             28.596359
longitude            77.249790
dtype: float64

In [27]:
#mean of indian restaurants for cluster#0 = 0.376542

In [37]:
dx1 = del_merged.loc[del_merged['Cluster Labels'] == 1]
dx1

,Neighborhood,Indian Restaurant,Cluster Labels,Borough,latitude,longitude
1,Alaknanda,0.076923,1,South Delhi,28.529336,77.251632
4,Ashok Vihar,0.111111,1,North West Delhi,28.699453,77.184826
10,Barakhamba Road,0.130952,1,New Delhi,28.629142,77.226149
14,Chandni Chowk,0.066667,1,North Delhi,28.643483,77.222742
14,Chandni Chowk,0.066667,1,Central Delhi,28.643483,77.222742
16,Chhattarpur,0.090909,1,South Delhi,28.507007,77.175417
17,Chittaranjan Park,0.100000,1,South Delhi,28.538752,77.249249
18,Civil Lines,0.142857,1,North Delhi,28.676857,77.225056
19,Connaught Place,0.126582,1,New Delhi,28.631383,77.219792
23,Defence Colony,0.150943,1,South Delhi,28.571791,77.232010


In [45]:
dx0['Neighborhood']

7           Badarpur
15     Chawri Bazaar
21         Daryaganj
33        Gautampuri
99             Okhla
106       Pitam Pura
Name: Neighborhood, dtype: object

In [29]:
dx1.mean()

Indian Restaurant     0.104003
Cluster Labels        1.000000
latitude             28.595567
longitude            77.201090
dtype: float64

In [44]:
dx1['Neighborhood']

1               Alaknanda
4             Ashok Vihar
10        Barakhamba Road
14          Chandni Chowk
14          Chandni Chowk
16            Chhattarpur
17      Chittaranjan Park
18            Civil Lines
19        Connaught Place
23         Defence Colony
26            Dhaula Kuan
26            Dhaula Kuan
30            Fateh Nagar
31         Friends Colony
34           Geeta Colony
35               Ghitorni
37             Golf Links
39        Greater Kailash
40             Green Park
41          Gulmohar Park
43      Hauz Khas Village
45              Inderpuri
49               Jangpura
53               Jor Bagh
55         Kailash Colony
57            Kamla Nagar
61            Khan Market
63         Khirki Village
66            Kirti Nagar
68          Kohat Enclave
71            Lahori Gate
72           Lajpat Nagar
73            Laxmi Nagar
74            Lodi Colony
75          Maharani Bagh
76             Mahipalpur
77          Malviya Nagar
80            Mayur Vihar
85          

In [30]:
#mean of indian restaurants for cluster#1 = 0.104003

In [31]:
dx2 =del_merged.loc[del_merged['Cluster Labels'] == 2]
dx2.head(10)

,Neighborhood,Indian Restaurant,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,0.250000,2,North West Delhi,28.614192,77.071541
2,Anand Vihar,0.153846,2,East Delhi,28.625691,77.101941
5,Azadpur,0.250000,2,North West Delhi,28.707657,77.175547
9,Bara Hindu Rao,0.250000,2,North Delhi,28.659518,77.205010
9,Bara Hindu Rao,0.250000,2,Central Delhi,28.659518,77.205010
13,Chanakyapuri,0.166667,2,New Delhi,28.594678,77.188521
20,Dariba Kalan,0.241379,2,North Delhi,28.654602,77.233379
27,Dilshad Garden,0.250000,2,North East Delhi,28.680682,77.322064
36,Gole Market,0.250000,2,New Delhi,28.633719,77.205627
42,Hauz Khas,0.193548,2,South Delhi,28.544256,77.206707


In [46]:
dx2['Neighborhood']

0        Adarsh Nagar
2         Anand Vihar
5             Azadpur
9      Bara Hindu Rao
9      Bara Hindu Rao
13       Chanakyapuri
20       Dariba Kalan
27     Dilshad Garden
36        Gole Market
42          Hauz Khas
44         INA Colony
50             Jasola
51        Jhandewalan
58         Karol Bagh
60       Keshav Puram
62            Khanpur
69            Kotwali
70      Krishna Nagar
89            Naraina
101             Palam
103     Paschim Vihar
105        Patparganj
120       Sainik Farm
126      Sarita Vihar
127    Sarojini Nagar
127    Sarojini Nagar
Name: Neighborhood, dtype: object

In [32]:
dx2.mean()

Indian Restaurant     0.202319
Cluster Labels        2.000000
latitude             28.610520
longitude            77.199586
dtype: float64

In [33]:
#mean of indian restaurants for cluster#2 = 0.202319

In [34]:
dx3 = del_merged.loc[del_merged['Cluster Labels'] == 3]
dx3.head(10)


,Neighborhood,Indian Restaurant,Cluster Labels,Borough,latitude,longitude
3,Ashok Nagar,0.000000,3,West Delhi,39.031714,-90.261223
6,Babarpur,0.000000,3,North East Delhi,28.687430,77.279755
8,Bali Nagar,0.040000,3,West Delhi,28.654138,77.128178
11,Bawana,0.000000,3,North West Delhi,28.799660,77.032885
12,Brij Puri,0.000000,3,East Delhi,28.702509,77.273777
22,Dayanand Colony,0.052632,3,South Delhi,28.562200,77.247613
24,Delhi Cantonment,0.000000,3,South West Delhi,28.583536,77.125985
25,Dhaka,0.000000,3,North West Delhi,39.031714,-90.261223
28,East Vinod Nagar,0.000000,3,East Delhi,28.622766,77.307504
29,East of Kailash,0.046512,3,South Delhi,28.557032,77.244614


In [35]:
dx3.mean()

Indian Restaurant     0.003222
Cluster Labels        3.000000
latitude             30.188669
longitude            52.210808
dtype: float64

In [54]:
dx3['Neighborhood']


3             Ashok Nagar
6                Babarpur
8              Bali Nagar
11                 Bawana
12              Brij Puri
22        Dayanand Colony
24       Delhi Cantonment
25                  Dhaka
28       East Vinod Nagar
29        East of Kailash
32           Gandhi Nagar
38             Govindpuri
46           Jahangirpuri
47                Jaitpur
48              Janakpuri
52         Jhilmil Colony
54            Kabir Nagar
56                Kalkaji
59          Kashmiri Gate
64          Kingsway Camp
65            Kirby Place
67     Kishangarh Village
78                Mandoli
79                Maujpur
81             Meera Bagh
82               Mehrauli
83             Model Town
84              Moti Bagh
87              Najafgarh
88             Nand Nagri
90                 Narela
91        Naveen Shahdara
92             Neeti Bagh
93            Nehru Nagar
95            Nehru Vihar
102          Pandav Nagar
108          Pratap Nagar
109           Preet Vihar
110         

In [36]:
#mean of indian restaurants for cluster#3 = 0.003222

# From the above analysis we can see that the least number of Indian Restaurants exist in the neighborhoods of cluster 3 (marked with YELLOW on the map).

# Hence it is advised to open an Indian Restaurants in a neighborhood of cluster 0 for maximium profit.